# WORDLE Solver

In [85]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from tqdm.notebook import tqdm_notebook
from scipy.stats import entropy
import threading
import time

In [59]:
VOCAB_FILES = ["data/wordle-guesses.txt", "data/wordle-answers.txt"]

In [60]:
def load_vocab(filepaths):
    vocab = set()
    for filepath in filepaths:
        with open(filepath) as f:
            for line in f:
                vocab.add(line.strip())
            
    vocab_list = list(vocab)
    vocab_to_idx = {w:i for i, w in enumerate(vocab_list)}
    return vocab_list, vocab_to_idx

In [61]:
vocab_list, vocab_to_idx = load_vocab(VOCAB_FILES)
print(len(vocab_list))

12972


## Hint Representation

We need a function that produces the hint string given a guess word and a goal word.

Hints take the form of an n-length array where n is the length of the guess word. Hints are created with the following rules (Let $c_i$ be the $i$th character of the guess word):

1. The $i$th position in the hint contains a 0 if $c_i$ does not appear in the goal word or if the $i$-length prefix of the guess word contains at least $n$ instances of $c_i$, where $n$ is the number of times $c_i$ appears in the goal word.

2. The $i$th position in the hint contains a 1 if $c_i$ appears in the goal word, but not in position $i$. 

3. The $i$th position in the hint contains a 2 if the $i$th position of the goal word is $c_i$

In [62]:
def get_hint(guess_word, goal_word):
    hint = np.zeros(len(guess_word), dtype=int)
    
    guess_counts = defaultdict(int)
    goal_counts = Counter(goal_word)
    
    for i, c in enumerate(guess_word):
        guess_counts[c] += 1
        
        if goal_counts[c] < guess_counts[c]:
            hint[i] = 0
            continue
        
        if c != goal_word[i]:
            hint[i] = 1
            continue
            
        hint[i] = 2
        
    return hint

In [63]:
example_hint = get_hint("aroma", "alter")
example_hint

array([2, 1, 0, 0, 0])

In [64]:
def hint_to_str(hint):
    chars = ["_", "o", "x"]
    hint_str = "".join([chars[idx] for idx in hint])
    return hint_str

In [65]:
def str_to_hint(hint_str):
    d = {'_':0, 'o':1, 'x':2}
    hint = np.zeros(len(hint_str), dtype=int)
    for i, c in enumerate(hint_str):
        hint[i] = d[c]
    return hint

In [66]:
hint_str = hint_to_str(example_hint)
hint_str

'xo___'

In [67]:
str_to_hint(hint_str)

array([2, 1, 0, 0, 0])

In [68]:
def hint_to_num(hint, base=3):
    hint_num = 0
    factor = 1
    for i, v in enumerate(hint):
        hint_num += v * factor
        factor *= base
    return hint_num

In [69]:
def num_to_hint(hint_num, base=3, sz=5):
    hint = np.zeros(sz, dtype=int)
    factor = base**(sz-1)
    for i in range(sz-1, -1, -1):
        digit = 0
        while (hint_num >= digit * factor) and (digit < base): 
            digit += 1
        hint[i] = digit - 1
        hint_num -= (digit - 1) * factor
        factor /= base
    return hint

In [70]:
hint_num = hint_to_num(example_hint)
hint_num

5

In [71]:
num_to_hint(hint_num)

array([2, 1, 0, 0, 0])

Since constructing the hint for a particular word combination can become costly when done repeatitively, it is faster to pre-load a "hint-matrix" that stores the results of evaluating all possible guess and goal word combinations.

In [72]:
def build_hint_matrix(vocab_list=None, load=None, save=None):
    
    if load is not None:
        hint_matrix = np.load(load)
    
    else:
        n_words = len(vocab_list)
        hint_matrix = np.zeros((n_words, n_words), dtype=int)

        hint_dict = {}        
        for i, guess_word in tqdm_notebook(enumerate(vocab_list), total=n_words):
            for j, goal_word in enumerate(vocab_list):
                hint = get_hint(guess_word, goal_word)
                hint_bytes = hint.tobytes()
                if hint_bytes not in hint_dict:
                    hint_dict[hint_bytes] = hint_to_num(hint)
                hint_matrix[i,j] = hint_dict[hint_bytes]
    
    if save is not None:
        np.save(save, hint_matrix)
    
    return hint_matrix

In [73]:
def build_hint_matrix_fast(vocab_list, n_threads=2):
    n_words = len(vocab_list)
    hint_matrix = np.zeros((n_words, n_words), dtype=int)
    
    hint_dict = {}
    def fill_rows(start, end):
        for i, guess_word in tqdm_notebook(enumerate(vocab_list[start:end]), total=end-start):
            for j, goal_word in enumerate(vocab_list):
                hint = get_hint(guess_word, goal_word)
                hint_bytes = hint.tobytes()
                if hint_bytes not in hint_dict:
                    hint_dict[hint_bytes] = hint_to_num(hint)
                hint_matrix[start + i,j] = hint_dict[hint_bytes]
                
    section_sz = int(n_words / n_threads) + 1
    threads = []
    for start in range(0, n_words, section_sz):
        end = start + section_sz if start + section_sz < n_words else n_words
        t = threading.Thread(target=fill_rows, args=(start, end))
        t.start()
        
    for t in threads:
        t.join()
    
    return hint_matrix

In [74]:
hint_matrix = build_hint_matrix(vocab_list=vocab_list, save="data/hint_matrix.npy")

  0%|          | 0/12972 [00:00<?, ?it/s]

## Calculating Word Entropy

Each potential guess word is associated with a distribution of possible hints (assuming that we select a goal word uniformly at random). The simplest approach is to assign an entropy value to each guess word according to its hint distribution. A more sophisticated approach takes into account the distribution of entropies conditioned on each possible hint that could be recieved after guessing. We can perform this evaluation recursively in order to determine the expected entropy of a set of word choices rather than a single choice.

In [181]:
def sort_by_entropy(word_idxs, entropies):
    sorted_args = np.argsort(entropies)[::-1]
    return word_idxs[sorted_args], entropies[sorted_args]

In [210]:
def get_entropies(hint_matrix, word_idxs=None, sort=False):
    word_idxs = pd.unique(word_idxs) if word_idxs is not None else np.arange(hint_matrix.shape[0])
    entropies = np.zeros_like(word_idxs, dtype=float)
    for i in range(word_idxs.shape[0]):
        word_idx = word_idxs[i]
        hint_counts = np.bincount(hint_matrix[word_idx])
        entropies[i] = entropy(hint_counts)
    
    if sort:
        word_idxs, entropies = sort_by_entropy(word_idxs, entropies)

    return word_idxs, entropies

In [183]:
word_idxs, entropies = get_entropies(hint_matrix, sort=True)
entropies

array([4.29339006, 4.26279884, 4.23813968, ..., 1.37084733, 1.30764523,
       1.30384486])

In [190]:
def display_word_entropies(word_idxs, entropies, cutoff=50):
    word_idxs, entropies = sort_by_entropy(word_idxs, entropies)
    print(f" Word | Entropy")
    print(f"------+---------")
    for i in range(min(cutoff, word_idxs.shape[0])):
        word_idx = word_idxs[i]
        word = vocab_list[word_idx]
        entropy_val = entropies[i]
        print(f"{word} | {entropy_val:0.5f}")

In [191]:
display_word_entropies(word_idxs, entropies)

 Word | Entropy
------+---------
tares | 4.29339
lares | 4.26280
rales | 4.23814
rates | 4.22559
teras | 4.21199
nares | 4.20521
soare | 4.20144
tales | 4.19700
reais | 4.19339
tears | 4.18130
arles | 4.17944
tores | 4.17156
salet | 4.17056
aeros | 4.16823
dares | 4.16605
saner | 4.15837
reals | 4.15830
lears | 4.15111
lores | 4.14292
serai | 4.14060
lanes | 4.13899
laers | 4.13739
pares | 4.13625
cares | 4.13560
tires | 4.13349
saine | 4.13299
seral | 4.12672
mares | 4.12546
reans | 4.12479
aloes | 4.12056
sared | 4.11910
roles | 4.11865
teals | 4.11613
aures | 4.11047
earls | 4.10802
taels | 4.10409
raise | 4.10325
tries | 4.10288
rotes | 4.10152
sorel | 4.10113
leats | 4.09809
nears | 4.09319
toeas | 4.09305
strae | 4.08800
rones | 4.08691
nates | 4.08649
earns | 4.07899
taser | 4.07798
toles | 4.07752
dales | 4.07735


### Recursive Entropy

Although the unconditional entropy of each guess word is a good indicator for how it will perform, the unconditional entropy only evaluates how well a guess will narrow down the set of solution words. It is possible that an initial guess word with a high entropy will result in all second guess-words having a low entropy. Therefore, rather than maximizing the entropy of the initial guess word, it is even more optimal to evaluate the entropy of a series of guesses. The entropy of each successive guess is conditioned on the the previous guesses and the entropies are summed in order to get the joint entropy.

In [216]:
def get_recursive_max_entropies(hint_matrix, word_idxs=None, max_depth=None, curr_depth=0, hard_mode=True, progress_bar=False, sort=False):
    word_idxs, entropies = get_entropies(hint_matrix, word_idxs) # these are the unconditional entropies of each word
    n_guesses = word_idxs.shape[0]
    n_solutions = hint_matrix.shape[1]
    
    if (max_depth is None) or (curr_depth < max_depth):
        for i in tqdm_notebook(range(n_guesses)) if progress_bar else range(n_guesses):
            word_idx = word_idxs[i]
            hint_ids, hint_counts = np.unique(hint_matrix[curr_idx], return_counts=True)
            hint_distribution = hint_counts / n_solutions
            n_hints = hint_ids.shape[0]

            max_entropies = np.zeros(n_hints, dtype=float)
            for j in range(n_hints):
                print(f"\r~~~~~~~~~~ Evaluating hint {j+1}/{n_hints} at depth {curr_depth+1}/{max_depth} ~~~~~~~~~~", end="")
                hint_id = hint_ids[j] # assume that this is the hint received in response to our current guess word
                viable_solutions = hint_matrix[i] == hint_id
                next_matrix = hint_matrix[:, viable_solutions] # takes only the columns of the hint matrix with viable goal words
                next_idxs = viable_solutions.nonzero()[0] if hard_mode else None
                _, next_entropies = get_recursive_max_entropies(next_matrix, next_idxs, max_depth, curr_depth+1, hard_mode, sort=False)
                max_entropies[j] = 0. if next_entropies.shape[0] == 0 else np.max(next_entropies)

            entropies[i] += np.sum(max_entropies * hint_distribution)
        
    if sort:
        word_idxs, entropies = sort_by_entropy(word_idxs, entropies)
    
    return idxs, entropies

In [217]:
word_idxs, deep_entropies = get_recursive_max_entropies(hint_matrix, idxs[:50], max_depth=1, progress_bar=True)

  0%|          | 0/50 [00:00<?, ?it/s]

~~~~~~~~~~ Evaluating hint 189/189 at depth 1/1 ~~~~~~~~~~

In [218]:
display_word_entropies(word_idxs, deep_entropies)

 Word | Entropy
------+---------
tares | 7.14560
arles | 7.07825
cares | 7.04644
rones | 7.03413
nares | 7.02628
rates | 7.01536
tores | 6.97134
roles | 6.96526
aures | 6.89405
pares | 6.89024
reans | 6.88279
laers | 6.87883
tries | 6.85827
serai | 6.82277
reais | 6.81978
mares | 6.81436
earns | 6.80884
toles | 6.80737
aeros | 6.77245
tears | 6.73486
reals | 6.72963
taser | 6.72768
lares | 6.72716
leats | 6.70785
rotes | 6.69841
soare | 6.66942
strae | 6.63430
tales | 6.62036
teals | 6.56793
dales | 6.55376
sorel | 6.51172
salet | 6.49798
rales | 6.25270
saner | 6.21693
lores | 6.16370
saine | 6.15980
dares | 6.15869
lanes | 6.12102
nates | 6.11480
teras | 6.10602
earls | 6.04947
lears | 6.04854
raise | 6.02050
sared | 6.00368
aloes | 5.96444
toeas | 5.95614
nears | 5.95606
tires | 5.92112
taels | 5.73873
seral | 5.47017


## WORDLE REPL

To make a interacting with this code a little easier, we can create repl to run while solving the wordle. Since the recursive calculation of the joint entropies is slow we are only using the unconditional entropy of the initial guess word.

In [219]:
GOAL_WORD = None
HARD_MODE = True

state = "suggesting"
hint_matrix_cpy = hint_matrix.copy()
remaining_words = np.arange(hint_matrix_cpy.shape[1])
while True:
    if state == "guessing":
        guess = input("Enter a guess word: ")
        while True:
            if guess not in vocab_to_idx:
                print(f"\"{guess}\" is not in the vocab list, try another word")
                guess = input("Enter a new guess word: ")
            elif HARD_MODE and vocab_to_idx[guess] not in remaining_words:
                print(f"\"{guess}\" is not a viable guess in hard-mode")
                guess = input("Enter a new guess word: ")
            else:
                break
        guess_idx = vocab_to_idx[guess]
        state = "hinting"
    
    if state == "hinting":
        if GOAL_WORD is None:
            hint_str = input("Enter the resulting hint: ")    
            hint = str_to_hint(hint_str)
        else:
            hint = get_hint(guess, GOAL_WORD)
            print(f"The hint is \"{hint_to_str(hint)}\"")
        hint_num = hint_to_num(hint)
        valid_words = hint_matrix_cpy[guess_idx] == hint_num
        remaining_words = remaining_words[valid_words]
        hint_matrix_cpy = hint_matrix_cpy[:, valid_words]
        state = "suggesting"
        print()
    
    if state == "suggesting":
        word_idxs = remaining_words if HARD_MODE else None
        word_idxs, entropies = get_entropies(hint_matrix_cpy, word_idxs)
        if remaining_words.shape[0] == 0:
            print("There are no more possible words in this vocab list")
            break
        if remaining_words.shape[0] == 1:
            remaining_word = vocab_list[remaining_words[0]]
            print(f"The goal word is \"{remaining_word}\"")
            break
        else:
            print("Here are the best guess words")
            display_word_entropies(word_idxs, entropies)
            print()
        state = "guessing"

Here are best guess words
 Word | Entropy
------+---------
tares | 4.29339
lares | 4.26280
rales | 4.23814
rates | 4.22559
teras | 4.21199
nares | 4.20521
soare | 4.20144
tales | 4.19700
reais | 4.19339
tears | 4.18130
arles | 4.17944
tores | 4.17156
salet | 4.17056
aeros | 4.16823
dares | 4.16605
saner | 4.15837
reals | 4.15830
lears | 4.15111
lores | 4.14292
serai | 4.14060
lanes | 4.13899
laers | 4.13739
pares | 4.13625
cares | 4.13560
tires | 4.13349
saine | 4.13299
seral | 4.12672
mares | 4.12546
reans | 4.12479
aloes | 4.12056
sared | 4.11910
roles | 4.11865
teals | 4.11613
aures | 4.11047
earls | 4.10802
taels | 4.10409
raise | 4.10325
tries | 4.10288
rotes | 4.10152
sorel | 4.10113
leats | 4.09809
nears | 4.09319
toeas | 4.09305
strae | 4.08800
rones | 4.08691
nates | 4.08649
earns | 4.07899
taser | 4.07798
toles | 4.07752
dales | 4.07735



Enter a guess word:  tares
Enter the resulting hint:  __o__



Here are best guess words
 Word | Entropy
------+---------
prion | 3.49912
groin | 3.48511
courd | 3.47519
droil | 3.47314
brond | 3.45564
proin | 3.44882
crony | 3.43502
irony | 3.42428
drony | 3.41796
crudo | 3.41567
bourn | 3.38581
bruin | 3.38427
bourd | 3.38278
bronc | 3.37978
gourd | 3.37555
round | 3.37551
orcin | 3.37178
broil | 3.36792
courb | 3.33285
grind | 3.33058
yourn | 3.32693
mourn | 3.31706
croon | 3.31577
inorb | 3.31132
proud | 3.29118
proyn | 3.28680
cronk | 3.28490
frond | 3.27143
drouk | 3.26881
robin | 3.26861
croup | 3.26851
grund | 3.26735
crown | 3.25389
fiord | 3.25187
bourg | 3.24513
drown | 3.24240
prong | 3.23812
orpin | 3.23594
guiro | 3.23369
crowd | 3.23195
grody | 3.23138
briny | 3.22981
bring | 3.20751
drink | 3.20502
crudy | 3.20220
proul | 3.19420
drily | 3.19264
cromb | 3.18792
chord | 3.18696
brood | 3.18642



Enter a guess word:  prion
Enter the resulting hint:  _o___



Here are best guess words
 Word | Entropy
------+---------
blurb | 2.09473
rhumb | 1.97920
flurr | 1.81431
rugby | 1.74816
churl | 1.69878
rumly | 1.63263
churr | 1.58903
rubby | 1.56071
uhuru | 1.35798
ruggy | 1.35221
rummy | 1.35221
ruddy | 1.14371



Enter a guess word:  rhumb
Enter the resulting hint:  o_x_x



The goal word is "blurb"
